In [ ]:
import cv2
import numpy as np
import os
import glob
from PIL import Image
import re

In [ ]:
def cut_maps(map, min_x, max_x):
    '''
    function to convert feature maps into a video
    map (str): feature map image location
    
    returns: cut feature maps with desired locations
    '''

    #load images
    images = [file for file in glob.glob(map + '/*Conv*.png')]
    images = [text[:-9] for text in images]
    strip_txt = 'Conv1.png'
    images.sort(key=lambda f: int(re.sub('\D', '', f)))
    files = [x+strip_txt for x in images]
    print(files)
    i=0

    for fil in files:

        img = Image.open(fil)
        img.show()
        width, height = img.size
        left = 3534
        right = 4404
        top = 18
        bottom = 168

        img = img.crop((left, top, right, bottom))
        img.save(f'/content/cut_feat{i}.png')
        i += 1


In [ ]:
def merge_images(img_img, flow_img):
    '''
    glues flow and real image frames together
    '''

    width1, height1 = img_img.shape[0], img_img.shape[1]
    width2, height2 = flow_img.shape[0], flow_img.shape[1]

    result_width = width1 + width2
    result_height = max(height1, height2)

    img_img = Image.fromarray(img_img, 'RGB')
    flow_img = Image.fromarray(flow_img, 'RGB')

    result = Image.new('RGB', (result_width, result_height))
    result.paste(img_img, box=(0, 0))
    result.paste(flow_img, box=(width1, 0))
    
    return result

In [ ]:
def multiframes2vid(file_loc, save_loc, fps):
    '''
    stitches frames and converts frames to an avi video
    '''

    flow_files = [file for file in glob.glob(file_loc + '/flow_x*')]
    img_files = [file for file in glob.glob(file_loc + '/img_*')]

    frames = []

    for flow_file, img_file in zip(flow_files, img_files):

        #load
        img_img = cv2.imread(img_file)
        flow_img = cv2.imread(flow_file)

        #resize
        img_img = cv2.resize(img_img, (512, 512))
        flow_img = cv2.resize(flow_img, (512, 512))

        #color
        img_img = cv2.cvtColor(img_img, cv2.COLOR_BGR2RGB)
        flow_img = cv2.cvtColor(flow_img, cv2.COLOR_BGR2RGB)

        #stitch and convert color
        img = merge_images(img_img, flow_img)
        img = np.asarray(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        frames.append(img)

    size = frames[0].shape
    flow_save = os.path.join(save_loc, file_loc.split('/')[-1] + '_comb.avi')
    out = cv2.VideoWriter(flow_save, cv2.VideoWriter_fourcc(*'DIVX'), fps, (1024, 512))

    for frame in frames:
        out.write(frame)
    out.release()

In [ ]:
def single_frames2vid(save_loc, fps):
    '''
    converts frames to an avi video
    '''
    frames = []
    for i in range(54):
        img = cv2.imread(f'/content/cut_feat{i}.png')
        img = cv2.resize(img, (1600, 200))
        frames.append(img)

    out = cv2.VideoWriter(save_loc, cv2.VideoWriter_fourcc(*'DIVX'), fps, (1600, 200))

    for frame in frames:
        out.write(frame)
    out.release()

In [ ]:
multiframes2vid('/content/drive/MyDrive/flow_test2/autumn/autumn3896', '/content/drive/MyDrive/flow_test2/finalvid', 16)

In [ ]:
single_frames2vid('/content/vid/videotime.avi',2)